# Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataframe = pd.read_csv('dataframe.tsv', sep='\t', encoding='utf-8', engine='python') #complete dataframe from milestone 1
adjacency = np.load('adjacency.npy') #complete undirected adjacency matrix

In [3]:
#trouver les articles à retirer: ceux qui sont isolés ou appartiennent au second component
connected_nodes = [0]
    
for node in connected_nodes:
    connections = adjacency[node,:]

    for i in range(len(connections)):
        if connections[i] == 1 and i not in connected_nodes:
            connected_nodes.append(i)    
            
connected_nodes.sort()

disconnected_nodes = []
j = 0
for i in connected_nodes:
    if i == 0:
        continue
    if i - j != 1:
        disconnected_nodes.append(i-1)
    j = i
disconnected_titles = []
for i in disconnected_nodes:
    disconnected_titles.append(dataframe.loc[i,'title'])

In [4]:
#reload everything
articles = pd.read_csv('data/articles.tsv', sep='\t', encoding='utf-8', engine='python', header=10)
articles.columns = ["title"]
categories = pd.read_csv('data/categories.tsv', sep='\t', encoding='utf-8', engine='python', header=11)
categories.columns = ["category"]
links = pd.read_csv('data/links.tsv', sep='\t', encoding='utf-8', engine='python', header=10)
links.columns = ["links"]

#completing categories with categoriless articles
for idx, rows in articles.iterrows():
    if articles.loc[idx,'title'] not in categories.index:
        new_row = pd.DataFrame(["None"], columns={"category"})
        new_row.index = [articles.loc[idx,'title']]
        categories = categories.append(new_row)
categories.sort_index(inplace=True)

#remove the disconnected articles from articles, categories and links
articles = articles.drop(disconnected_nodes)
articles.reset_index(level=0, inplace=True, drop=True)

categories = categories.drop(disconnected_titles)

for title in disconnected_titles:
    if title in links.index:
        links = links.drop(title)

In [5]:
#creating the dataframe (code from milestone 1)
dataframe = categories
dataframe.reset_index(level=0, inplace=True)
dataframe = dataframe.rename(columns={'index':'title'})
dataframe.reset_index(level=0, inplace=True)
dataframe = dataframe.rename(columns={'index':'cat_idx'})

cur_t = dataframe.loc[0, 'title']
for idx, row in dataframe.iterrows():
    if idx == 0:
        continue #ignore first row
    if dataframe.loc[idx, 'title'] == cur_t:
        dataframe.loc[idx, 'cat_idx'] = dataframe.loc[idx-1, 'cat_idx'] + 1
    else:
        cur_t = dataframe.loc[idx, 'title']
        dataframe.loc[idx, 'cat_idx'] = 0
        
dataframe = dataframe.pivot(index='title', columns='cat_idx', values='category')
dataframe = dataframe.rename(columns={0:'Label1'})
dataframe = dataframe.rename(columns={1:'Label2'})
dataframe = dataframe.rename(columns={2:'Label3'})

In [6]:
#directed adjacency matrix creation
articles.reset_index(level=0, inplace=True)
articles = articles.rename(columns={'index':'node_idx'})
art_to_idx = articles[['node_idx', 'title']]
art_to_idx = art_to_idx.set_index('title')

links.reset_index(level=0, inplace=True)
links = links.join(art_to_idx, on='index')
links = links.join(art_to_idx, on='links', rsuffix='_parent')
links = links.drop(columns=['index','links'])

n_nodes = len(articles)
adjacency = np.zeros((n_nodes, n_nodes), dtype=int)

for idx, row in links.iterrows():
    if np.isnan(row.node_idx_parent):
        continue
    i, j = int(row.node_idx), int(row.node_idx_parent)
    adjacency[i, j] = 1

In [7]:
dataframe.to_csv('dataframe_final.tsv', sep='\t', encoding='utf-8')
np.save('adjacency_final.npy', adjacency)